In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df_rating = pd.read_csv('/content/drive/MyDrive/Books/Ratings.csv')
df_books = pd.read_csv('/content/drive/MyDrive/Books/Books.csv')
df_users = pd.read_csv('/content/drive/MyDrive/Books/Users.csv')
df_books.head()
# df_rating.head()
# df_users.head()

<ipython-input-3-398e4bc70d4a>:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_books = pd.read_csv('/content/drive/MyDrive/Books/Books.csv')


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [4]:
df_images = df_books.iloc[:, -2:]
df_books = df_books.iloc[:, :-2]
# df_images.head()
df_books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...


In [5]:
df_rating

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [6]:
userCounts = df_rating['User-ID'].value_counts()
isbnCounts = df_rating['ISBN'].value_counts()
print(isbnCounts)

0971880107     2502
0316666343     1295
0385504209      883
0060928336      732
0312195516      723
               ... 
1568656386        1
1568656408        1
1569551553        1
1570081808        1
05162443314       1
Name: ISBN, Length: 340556, dtype: int64


In [7]:
len(df_rating['User-ID'].unique())

105283

In [8]:
dfr = df_rating
dfr = dfr[~dfr['ISBN'].isin(isbnCounts[isbnCounts < 100].index)]

dfr = pd.merge(right=dfr, left = df_books, on="ISBN")
dfr.head()
dfr = dfr[~dfr['User-ID'].isin(userCounts[userCounts < 200].index)]
dfr = dfr.drop_duplicates(["Book-Title", "User-ID"])

In [9]:
combine_book_rating = dfr
columns = ['Year-Of-Publication', 'Book-Author', 'Image-URL-S', 'Publisher']
combine_book_rating =  combine_book_rating.drop(columns, axis=1)
combine_book_rating.head()

,ISBN,Book-Title,User-ID,Book-Rating
0,0440234743,The Testament,277478,0
5,0440234743,The Testament,2977,0
7,0440234743,The Testament,3363,0
10,0440234743,The Testament,7346,9
14,0440234743,The Testament,9856,0


In [10]:
df_for_nn = combine_book_rating
combine_book_rating.shape[0]

49136

In [ ]:
# df_ready = df_for_nn.pivot_table(index=['User-ID'],columns=['ISBN'],values='Book-Rating').fillna(0).T
# df_ready.head()

In [ ]:
# df_ready.index = df_ready.join(df_books.set_index('ISBN'))['Book-Title']
# df_ready

In [11]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Concatenate, Dense
from tensorflow.keras.models import Model

In [12]:
df_for_nn.head()

,ISBN,Book-Title,User-ID,Book-Rating
0,0440234743,The Testament,277478,0
5,0440234743,The Testament,2977,0
7,0440234743,The Testament,3363,0
10,0440234743,The Testament,7346,9
14,0440234743,The Testament,9856,0


In [13]:
unique_books = df_for_nn['Book-Title'].unique()
book_id_mapping = {book_title: book_id for book_id, book_title in enumerate(unique_books, start=1)}

df_for_nn['Book-ID'] = df_for_nn['Book-Title'].map(book_id_mapping)

df_for_nn

,ISBN,Book-Title,User-ID,Book-Rating,Book-ID
0,0440234743,The Testament,277478,0,1
5,0440234743,The Testament,2977,0,1
7,0440234743,The Testament,3363,0,1
10,0440234743,The Testament,7346,9,1
14,0440234743,The Testament,9856,0,1
...,...,...,...,...,...
137411,0515135739,Eleventh Hour: An FBI Thriller (FBI Thriller (...,236283,0,673
137416,0515135739,Eleventh Hour: An FBI Thriller (FBI Thriller (...,251613,0,673
137417,0515135739,Eleventh Hour: An FBI Thriller (FBI Thriller (...,252071,0,673
137420,0515135739,Eleventh Hour: An FBI Thriller (FBI Thriller (...,256407,0,673


In [14]:
print(len(df_for_nn['User-ID'].unique())+1)
print(df_for_nn['User-ID'].max())
print(df_for_nn['Book-ID'].max())

889
278418
673


In [15]:
# Define input layers
user_input = Input(shape=[1], name='user_input')
book_input = Input(shape=[1], name='book_input')

user_embedding = Embedding(input_dim=278419, output_dim=10)(user_input)
book_embedding = Embedding(input_dim=674, output_dim=10)(book_input)

user_flat = Flatten()(user_embedding)
book_flat = Flatten()(book_embedding)

prod = Dot(name="Dot-Product", axes=1)([book_flat, user_flat])
model = Model([user_input, book_input], prod)
model.compile('adam', 'mean_squared_error')

In [ ]:
history = model.fit([df_for_nn['User-ID'], df_for_nn['Book-ID']], df_for_nn['Book-Rating'], epochs=10, verbose=1)

In [16]:
import pickle

In [ ]:
with open('/content/drive/MyDrive/Neural Network/model(O)', 'wb') as file:
    pickle.dump(model, file)

In [18]:
with open('/content/drive/MyDrive/Neural Network/model(O)', 'rb') as file:
    model = pickle.load(file)

In [19]:
"The Testament" in df_for_nn["Book-Title"].values

True

In [46]:
# book_data = np.array(list(set(df_for_nn['Book-ID'])))
# user = np.array([1 for i in range(len(book_data))])
# predictions = model.predict([user, book_data])
# predictions = np.array([a[0] for a in predictions])
# recommended_book_ids = (-predictions).argsort()[:5]
# print(recommended_book_ids)
# print(predictions[recommended_book_ids])
# output_df=df_for_nn[df_for_nn['Book-ID'].isin(recommended_book_ids)]
# book_titles = output_df['Book-Title'].values
# print(set(book_titles.tolist()))

In [44]:
def get_recommendations(book_title = ""):
  book_id = df_for_nn.loc[df_for_nn['Book-Title'] == book_title, 'Book-ID'].iloc[0]
  # print(book_id)
  n = 6
  book_data = np.full(n,book_id)
  # print(book_data)
  user_id = 1
  user = np.full(shape=(n, 1), fill_value=user_id)
  predictions = model.predict([user, book_data])
  # print(predictions)
  predictions = np.array([a[0] for a in predictions])
  recommended_book_ids = (-predictions).argsort()[:n]
  # print(recommended_book_ids)
  print(predictions[recommended_book_ids])
  output_df=df_for_nn[df_for_nn['Book-ID'].isin(recommended_book_ids)]
  book_titles = output_df['Book-Title'].values
  return set(book_titles.tolist())

In [45]:
get_recommendations("Interview with the Vampire")

1/1 [==============================] - 0s 23ms/step
[0.01955714 0.01955714 0.01955714 0.01955714 0.01955714 0.01955714]


{'Airframe',
 'Beloved (Plume Contemporary Fiction)',
 'The Testament',
 'Timeline',
 'Wild Animus'}